# Convolutional neural network core

In [ ]:
import tensorflow as tf
import random,csv
import matplotlib.pyplot as plt

In [ ]:
def next_batch(feature_list,label_list,size):
    feature_batch_temp=[]
    label_batch_temp=[]
    f_list = random.sample(range(len(feature_list)), size)
    for i in f_list:
        feature_batch_temp.append(feature_list[i])
    for i in f_list:
        label_batch_temp.append(label_list[i])
    return feature_batch_temp,label_batch_temp

In [ ]:
def weight_variable(shape,layer_name):
    # Define a weight shape tensor of a shape shape
    with tf.name_scope(layer_name + '_Weights'):
        Weights = tf.Variable(tf.truncated_normal(shape, stddev=0.1),name='W')
    tf.summary.histogram(layer_name + '_Weights', Weights)
    return Weights

In [ ]:
def bias_variable(shape,layer_name):
    #defined a shape shape of the bias tensor
    with tf.name_scope(layer_name + '_biases'):
        biases = tf.Variable(tf.constant(0.1, shape=shape),name='b')
    tf.summary.histogram(layer_name + '_biases', biases)
    return biases

In [ ]:
def conv2d(x, W,layer_name):
   #convolution calculation function
   # stride [1, x steps, y steps, 1]
   #padding:SAME/FULL/VALID (marginal processing)
    with tf.name_scope(layer_name + '_h_conv2d'):
        h_conv2d = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
    return h_conv2d

In [ ]:
def max_pool_2x2(x,layer_name):
    #max pooling function
    #ksize [1, x side length, y side length, 1] pooled window size
    #stride [1, x steps, y steps, 1]
    #padding:SAME/FULL/VALID (marginal processing)
    with tf.name_scope(layer_name + '_h_pool'):
        h_pool = tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    return h_pool

In [ ]:
def load_data():
    global feature
    global label
    global feature_full
    global label_full
    feature=[]
    label=[]
    feature_full=[]
    label_full=[]
    file_path ='kddcup/kddcup.data_10_percent_corrected_handled2.csv'
    with (open(file_path,'r')) as data_from:
        csv_reader=csv.reader(data_from)
        for i in csv_reader:
            # print (i)
            label_list=[0]*23
            feature.append(i[:36])
            label_list[int(i[41])]=1
            label.append(label_list)
            # print label
            # print feature
        csv_reader = csv.reader(data_from)
    file_path_full ='kddcup 2/kddcup.data.corrected_handled2.csv'
    with (open(file_path_full,'r')) as data_from_full:
        csv_reader_full=csv.reader(data_from_full)
        for j in csv_reader_full:
            # print i
            label_list_full=[0]*23
            feature_full.append(j[:36])
            label_list_full[int(j[41])]=1
            label_full.append(label_list_full)

In [ ]:
if __name__  == '__main__':
    global feature
    global label
    global feature_full
    global label_full
    history_step = []
    history_loss = []
    history_train_accuracy = []
    history_test_accuracy = []
    #load data
    load_data()
    feature_test = feature
    feature_train =feature_full
    label_test = label
    label_train = label_full
    #Define the placeholder to enter
    with tf.name_scope('inputs'):
        xs = tf.placeholder(tf.float32, [None, 36],name='pic_data') # 6x6
        ys = tf.placeholder(tf.float32, [None, 23],name='pic_label')
        keep_prob = tf.placeholder(tf.float32,name='keep_prob')
        x_image = tf.reshape(xs, [-1, 6, 6, 1])   

    #The first convolution layer
    with tf.name_scope('conv1_layer'):
        W_conv1 = weight_variable([3,3,1,32],layer_name='conv1')  # convolution kernel 3x3, input thickness 1, output thickness 32
        b_conv1 = bias_variable([32],layer_name='conv1')
        h_conv1 = tf.nn.sigmoid(conv2d(x_image, W_conv1,layer_name='conv1') + b_conv1)    # Output size: 6x6x32
        h_pool1 = max_pool_2x2(h_conv1,layer_name='conv1')                             # Output size: 3x3x32


   #The first full connection layer
   #with dropout
    with tf.name_scope('fc1_layer'):
        W_fc1 = weight_variable([3*3*32,1024],layer_name='fc1')
        b_fc1 = bias_variable([1024],layer_name='fc1')
        with tf.name_scope('reshape'):
            h_pool2_flat = tf.reshape(h_pool1, [-1,3*3*32])
        with tf.name_scope('sigmoid'):
            h_fc1 = tf.nn.sigmoid(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
        with tf.name_scope('dropout'):
            h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
            
   #The second full connection layer
    with tf.name_scope('fc2_layer'):
        W_fc2 = weight_variable([1024, 23],layer_name='fc2')
        b_fc2 = bias_variable([23],layer_name='fc2')
        with tf.name_scope('softmax'):
            prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


    #Calculate loss/cost
    with tf.name_scope('loss'):
        cross_entropy = -tf.reduce_sum(ys * tf.log(prediction))       # loss
    tf.summary.scalar('loss',cross_entropy)
    #Calculate accuracy
    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(ys,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)
    #Use the Adam optimizer to achieve the steepest gradient drop
    with tf.name_scope('train'):
        train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

    with tf.Session() as sess:
        #Initialize all tensors
        sess.run(tf.initialize_all_variables())
        #Draw the neural network structure
        writer =tf.summary.FileWriter("multi_logs/", sess.graph)
        #Indicate the change in the tag value during training and evaluation
        merged =  tf.summary.merge_all()
        train_writer = tf.summary.FileWriter("multi_logs/train", sess.graph)
        test_writer = tf.summary.FileWriter("multi_logs/test", sess.graph)

        for step in range(501):
            feature_train_batch, label_train_batch = next_batch(feature_train, label_train,1000)  #Random gradient descent training, each time you choose a batch of 1000
            feature_test_batch, label_test_batch = next_batch(feature_test, label_test,1000)  
            sess.run(train_step, feed_dict={xs: feature_train_batch, ys: label_train_batch, keep_prob: 0.5})
            if step % 50 == 0:
                train_writer.add_summary(sess.run(merged, feed_dict={xs: feature_train_batch, ys: label_train_batch, keep_prob: 1}), step)
                test_writer.add_summary(sess.run(merged, feed_dict={xs: feature_test, ys: label_test, keep_prob: 1}), step)
                history_step.append(step)
                history_loss.append(sess.run(cross_entropy, feed_dict={xs: feature_train_batch, ys: label_train_batch, keep_prob: 1}))
                history_test_accuracy.append(sess.run(accuracy, feed_dict={xs: feature_test, ys: label_test, keep_prob: 1}))
                history_train_accuracy.append(sess.run(accuracy, feed_dict={xs: feature_train_batch, ys: label_train_batch, keep_prob: 1}))
                print(step)


In [ ]:

# allow use of interactive plot controls, e.g., zoom
get_ipython().run_line_magic('matplotlib', 'notebook')
plt.figure()
plt.subplot(1, 3, 1)
plt.title('loss')
plt.plot(history_step, history_loss)

plt.subplot(1,3,2)
plt.title('train_accuracy')
plt.plot(history_step, history_train_accuracy)

plt.subplot(1,3,3)
plt.title('test_accuracy')
plt.plot(history_step, history_test_accuracy)
plt.show()